In [1]:
import numpy as np
import matplotlib.pyplot as plt
import openmc
import openmc.mgxs as mgxs

In [2]:
materials = openmc.Materials.from_xml("infmed_materials.xml") 

In [3]:
geometry = openmc.Geometry.from_xml('infmed_geometry.xml')

/home/robertsj/opt/miniconda3/lib/python3.9/site-packages/openmc/mixin.py:67: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/robertsj/opt/miniconda3/lib/python3.9/site-packages/openmc/mixin.py:67: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/robertsj/opt/miniconda3/lib/python3.9/site-packages/openmc/mixin.py:67: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)


In [4]:
settings = openmc.Settings.from_xml("slowingdown_settings.xml")
settings.run_mode = 'eigenvalue'

In [5]:
# Instantiate a 2-group EnergyGroups object
groups = mgxs.EnergyGroups()
groups.group_edges = np.array([1e-5, 0.625, 10.0e6])

# Instantiate a few different sections
cell = geometry.get_all_cells()[1]
total = mgxs.TotalXS(domain=cell, groups=groups)
absorption = mgxs.AbsorptionXS(domain=cell, groups=groups)
nufission = mgxs.FissionXS(domain=cell, groups=groups, nu=True)
scattering = mgxs.ScatterXS(domain=cell, groups=groups, nu=True)
scattering = mgxs.ScatterMatrixXS(domain=cell, groups=groups)
scattering.correction = None 
chi = mgxs.Chi(domain=cell, groups=groups)

tallies = openmc.Tallies()
tallies += total.tallies.values()
tallies += nufission.tallies.values()
tallies += absorption.tallies.values()
tallies += scattering.tallies.values()
tallies += chi.tallies.values()

tallies.export_to_xml('effxs_settings.xml')

/home/robertsj/opt/miniconda3/lib/python3.9/site-packages/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=1.
  warn(msg, IDWarning)
/home/robertsj/opt/miniconda3/lib/python3.9/site-packages/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/home/robertsj/opt/miniconda3/lib/python3.9/site-packages/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=9.
  warn(msg, IDWarning)


In [6]:
# Write all needed xml files to their default names
for xml in [materials, geometry, settings, tallies]:
    xml.export_to_xml()
    
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [7]:
sp = openmc.StatePoint('statepoint.10.h5')
# Load the tallies from the statepoint into each MGXS object
total.load_from_statepoint(sp)
absorption.load_from_statepoint(sp)
nufission.load_from_statepoint(sp)
scattering.load_from_statepoint(sp)
chi.load_from_statepoint(sp)
# Close the statepoint file now that we're done getting info
sp.close()

In [8]:
total.print_xs()

Multi-Group XS
	Reaction Type  =	total
	Domain Type    =	cell
	Domain ID      =	1
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 10000000.0eV]:	6.28e-01 +/- 3.71e+00%
            Group 2 [1e-05      - 0.625     eV]:	1.28e+00 +/- 1.08e+00%





In [9]:
scattering.print_xs()

Multi-Group XS
	Reaction Type  =	scatter matrix (P0)
	Domain Type    =	cell
	Domain ID      =	1
	Energy Groups: 
            Group 1 [0.625      - 10000000.0eV]
            Group 2 [1e-05      - 0.625     eV]
	Nuclide        =	sum
	Cross Sections [cm^-1]:
            Group 1 -> Group 1:		5.95e-01 +/- 3.68e+00%
            Group 1 -> Group 2:		1.67e-02 +/- 3.76e+00%

            Group 2 -> Group 1:		7.91e-04 +/- 5.82e+00%
            Group 2 -> Group 2:		1.09e+00 +/- 1.08e+00%







In [10]:
T = np.diag(total.get_xs()); T

array([[0.62809368, 0.        ],
       [0.        , 1.28259656]])

In [11]:
S = scattering.get_xs().T; S

array([[5.94793459e-01, 7.91479532e-04],
       [1.66536741e-02, 1.08535587e+00]])

In [12]:
F = nufission.get_xs(); F

array([0.01372892, 0.35643866])

In [13]:
A = absorption.get_xs(); A

array([0.01650701, 0.19614989])

In [24]:
X = chi.get_xs();   X

array([0.99982656, 0.        ])

In [25]:
FF = np.outer(X, F); FF

array([[0.01372654, 0.35637684],
       [0.        , 0.        ]])

In [26]:
MM = T - S; MM

array([[ 0.03330022, -0.00079148],
       [-0.01665367,  0.19724069]])

In [27]:
np.linalg.solve(MM, X)

array([30.08500178,  2.54017473])

In [28]:
scattering.get_flux()

array([30.86071256,  2.60943251])

In [29]:
np.linalg.solve(MM, [1, 0]) / total.get_flux()

array([0.97480528, 0.97405991])

In [20]:
A@absorption.get_flux()*.83

0.8475543339876395

In [21]:
np.diag(A)@np.linalg.solve(MM, X)

array([0.49669945, 0.49834143])

In [22]:
(T-S)@np.linalg.solve(MM, X)

array([1.00000000e+00, 9.75973562e-18])

In [23]:
total.energy_groups.group_edges

array([1.00e-05, 6.25e-01, 1.00e+07])

In [31]:
np.linalg.eig(np.linalg.inv(T-S)@FF)

(array([-1.11022302e-16,  1.31845120e+00]),
 array([[-0.99925905, -0.99645446],
        [ 0.03848839, -0.0841339 ]]))

In [32]:
1/.084

11.904761904761903

In [33]:
30.86/2.6

11.869230769230768